# Basic Azure Resource Management

## PowerShell

PowerShell is a powerful scripting and automation framework developed by Microsoft. It is designed for task automation and configuration management and is particularly useful for managing and automating Microsoft Windows environments. PowerShell uses a command-line interface with a scriptable approach, and it's built on the .NET Framework.

* [What is PowerShell?](https://learn.microsoft.com/powershell/scripting/overview?view=powershell-7.3&WT.mc_id=AZ-MVP-5004796)

## PowerShell and Microsoft Azure

When it comes to Microsoft Azure, PowerShell provides a robust set of cmdlets (pronounced "command-lets") that enable you to interact with and manage Azure resources, making it a valuable tool for working with Azure services. 

When you run a PowerShell cmdlet to, for example, create a virtual machine or retrieve information about an Azure resource, the cmdlet translates your request into an HTTP request to the relevant Azure REST API endpoint.

* [Azure PowerShell Documentation](https://learn.microsoft.com/powershell/azure/?view=azps-10.4.1&WT.mc_id=AZ-MVP-5004796)

## Resource Creation

In [ ]:
# Import Module
Import-Module Az -Verbose

In [ ]:
#Create Azure Resource Group
New-AzResourceGroup -Name "MyResourceGroup" -Location "West US"

In [ ]:
# Get Regions

Get-AzLocation | Select-Object -First 1
Get-AzLocation | Select-Object DisplayName, Location, PhysicalLocation, GeographyGroup | Format-Table


In [ ]:
#Create Azure Resource Group
$region = 'AustraliaEast'
New-AzResourceGroup -Name "MyResourceGroup$region" -Location $region

In [ ]:
# Create a storage account (attempt1)

$region = 'AustraliaEast'
$ResourceGroupName = "MyResourceGroup$region"
New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystorageaccount" -Location $region -SkuName Standard_LRS -AllowBlobPublicAccess $false

In [ ]:
# Create a storage account (attempt2)
$uniqueId = [guid]::NewGuid().ToString().Substring(0,4)
$region = 'AustraliaEast'
$ResourceGroupName = "MyResourceGroup$region"
New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -Location $region -SkuName Standard_LRS -AllowBlobPublicAccess $false -verbose

In [ ]:
#Remove Azure Storage account
$region = 'AustraliaEast'
$ResourceGroupName = $ResourceGroupName
Remove-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -Force -verbose
Get-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -verbose

In [ ]:
# Create an Azure Virtual Network
$region = 'AustraliaEast'
$ResourceGroupName = 'network-prod-rg'
$VNetname = 'vnet-prod'
$subnetname = 'infraservers'
$subnetAddressPrefix = '10.0.0.0/24'

# Create a resource group
$ResourceGroup = Get-AzResourceGroup -Name $ResourceGroupName -ErrorAction SilentlyContinue

if ($null -eq $ResourceGroup)
{
    Write-Host "Creating Resource Group $ResourceGroupName in $region" -ForegroundColor Yellow
    $ResourceGroup = New-AzResourceGroup -Name $ResourceGroupName -Location $region -Force
}
else
{
    Write-Host "Resource Group $ResourceGroupName already exists in $region" -ForegroundColor Green
}

# Create a virtual network
$AzVNET = New-AzVirtualNetwork -ResourceGroupName $ResourceGroupName -Name $VNetname -AddressPrefix '10.0.0.0/16' -Location $region

# Create a subnet
$subnetConfig = Add-AzVirtualNetworkSubnetConfig -Name $subnetname -AddressPrefix $subnetAddressPrefix -VirtualNetwork $AzVNET



In [ ]:
# Get full object output
# Alias (This is a pipeline to the Format-List cmdlet (fl is an alias for Format-List). It formats the output as a list of properties for each object. This can make it easier to read the details of the virtual network.)

Get-AzVirtualNetwork -ResourceGroupName $ResourceGroupName -Name $VNetname # | fl

In [ ]:
# Alias

Get-Alias  | Select-Object -First 2

In [ ]:
#splat

$configData = @{
    ResourceGroupName = "MyResourceGroup"
    Location = "West US"
    StorageAccountName = "stgacctest100"
}

try {
    New-AzStorageAccount -ResourceGroupName $configData.ResourceGroupName -Name $configData.StorageAccountName -Location $configData.Location -SkuName Standard_LRS
} catch {
    Write-Error "Failed to create storage account: $_"
}

In [ ]:
#splat as parameter

$configData = @{
    "ResourceGroupName" = "MyResourceGroup"
    "Location" = "West US"
    "StorageAccountName" = "stgacctest100"
    "SkuName" = "Standard_LRS"

}

try {
    New-AzStorageAccount @configData
} catch {
    Write-Error "Failed to create storage account: $_"
}

In [ ]:
# Tags
$ResourceGroupName = 'TagTestRG'
New-AzResourceGroup -Name $ResourceGroupName -Location 'AustraliaEast'
Set-AzResourceGroup -Name $ResourceGroupName -Tag @{ Department = "Finance"; Project = "Q1" }

In [ ]:
# Get Tag Resource Group
$ResourceGroupName = 'TagTestRG'
(Get-AzResourceGroup -Name $ResourceGroupName).Tags

In [ ]:
$ResourceGroupName = 'TagTestRG'
$tags = (Get-AzResourceGroup -Name $ResourceGroupName).Tags
$tags.Remove("Project")
Set-AzResourceGroup -Name $ResourceGroupName -Tag $tags